# Q-learning without higher math

### Can one algorithm learn to solve many problems?

If I asked you to write a program to solve a maze, you might think about how a maze works, and ask *which way to the exit?* If I asked you write a program to play tic-tac-toe, you might start by thinking about the board, or the rules, or whether to go first.

But: what if I asked you to write a program that could learn to solve either a maze or tic-tac-toe, without changing a single line of code?

Does that sound possible?

### Q-learning

Let's take a journey from conventional programming toward machine learning. You will use a simple maze as a sample problem. By applying a algorithm called **Q-learning**, you won't just find the way through the maze. You will demonstrate a way to solve a whole class of problems, by creating programs that learn as they go.

You will be working in the field of **reinforcement learning**, a major branch of machine learning. RL confronts problem that evolve one step at a time, like playing a game, driving a car, or building spooky dog-shaped robots. Other major branches deal with problems like image or speech recognition. You will be playing games, not looking for cats. Looking for cats is a totally different problem.

### Calculus: not required

You won't need calculus, probability & statistics, or linear algebra to complete your journey. If you know those topics, that's great... those would allow you to solve problems with larger dimensions, like playing Super Mario Bros or chess. **However: the principles that guide Q-learning are exactly the same, with or without the higher math.** The purpose of the math is to make approximations when a problem's dimensions get out of hand. The core algorithm is unchanged.


In [0]:
#@title (run this cell to load machine learning environment) { vertical-output: true, display-mode: "form" }
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

class Maze():
    # nicknames for actions... for beginners, only
    actions = ['N','S','E','W']
    # offsets to move North, South, East, or West
    offset = [(-1,0),(1,0),(0,1),(0,-1)]

    def __init__(self):
        self.maze = np.array([[0,0,0,-1],[0,-1,0,0],[0,0,-1,0],[-1,-1,0,0]])  # the maze is hardcoded
        self.mark = None
        self.reset()

    # clear the blocks, leaving an empty maze
    def remove_blocks(self):
        self.maze = np.zeros((4,4),dtype=int)
    
    # reset the environment
    def reset(self, random=False):
        self.i = 1
        self.maze[0][0] = self.i  # mark initial position with counter
        self.player = (0,0)
        return self.player

    # take one step, using a specified action
    def step(self, action):
        self.i += 1
        if type(action) is str:
            action = Maze.actions.index(action)
        obs = tuple(np.add(self.player, Maze.offset[action]))
        if max(obs) > 3 or min(obs) < 0 or self.maze[obs] == -1:
            # player is out of bounds or square is blocked... don't move player
            return self.player, -1, True
        else:
            # move was successful, advance player to new position
            self.maze[obs] = self.i
            self.player = obs
            if np.array_equal(self.player, (3,3)):  # reached the exit
                return self.player, 1, True
            else:
                return self.player, 0, False        # no outcome (player is on an open space)

    # return a random action (equally distributed across the action space)
    def sample(self):
        return Maze.actions[np.random.randint(4)]

    # return a random action in numeric form (equally distributed across the action space)
    def sample_n(self):
        return np.random.randint(4)

    def action_space(self):
        return Maze.actions

    def action_space_n(self):
        return [0,1,2,3]

    def state_space(self):
        return 4,4

    def __str__(self):
        out = '\n=========='
        for x in range(4):
            out += '\n|'
            for y in range(4):
                if self.mark is not None and self.mark[0]==x and self.mark[1]==y:
                    out += '? '
                elif self.maze[x][y]>0:
                    out += str(self.maze[x][y]) + ' '
                elif self.maze[x][y]==-1:
                    out += 'X '
                elif x==3 and y==3:
                    out += '* '
                else:
                    out += '. '
            out += '|'
        out += '\n==========\n'
        return out

    def print_q(q, mode='all'):
        print('=====  ================================')
        print('state         N       S       E       W\n')
        for x in range(4):
            for y in range(4):
                out = '('
                out += str(x)
                out += ','
                out += str(y)
                out += ')  '
                for a in range(4):
                    if mode=='all':
                        out += '{:>8,d}'.format(int(q[x][y][a]))
                    elif mode=='rewards':
                        if q[m][n][a] > 0:
                            out += '{:8.3f}'.format(q[x][y][a])
                        else:
                            out += '        '
                print(out)
            print('-----  --------------------------------')

    def plot(q):
      fig = plt.figure(figsize=(16,6))
      ax1 = fig.add_subplot(121, projection='3d')
      x,y,b = [],[],[]
      z = np.zeros((4,4))
      for m in range(4):
        for n in range(4):
          x.append(m)
          y.append(n)
          b.append(0)
          z[m][n] += max(max(q[m][n]),0)

      ax1.bar3d(x, y, b, 1, 1, np.ravel(z), shade=True)
      plt.title('Positive Q-scores')
      plt.xlabel('state (row)')
      plt.ylabel('state (col)')
      plt.show()

print('the maze environment is all set to go')


# Let's have a look at the maze

### Machine learning environments

Many machine learning problems are packaged into **environments**. An environment is a program that presents a problem in a rigorous, predictable way. When you want to write an algorithm to solve more than one problem, environments are a big help, because you can take your algorithm and aim it at more than one problem.

Facilities like the **OpenAI Gym** are filled with machine learning environments. You can connect to any of them in more or less the same way. Take a look, [here](https://gym.openai.com/envs/#classic_control).

Your first environment represents a 4x4 maze. The maze is engineered just like the environments at OpenAI. Once you solve the maze, you can head over to the gym and take a crack at something more complex.

You can create and view the maze like this:

In [0]:
m = Maze()     # make an instance of the maze environment
print(m)       # print the maze

# Stepping through the maze

Like many environments, the maze provides functions to reset to an initial state, and to take one step:

```
reset()        # start over, from the beginning
step(action)   # take a step ('N','S','E','W')
```

You can take your very first step in the maze like this:


In [0]:
env = Maze()     # make an instance of the maze environment
env.reset()      # reset to initial state (always a good idea)
env.step('E')    # take a step to the east...
print(env)       # ...where am I now?

Notice the numbering that goes from (1) to (2)? That's you, taking one step to the east.

Since you know a thing or two about mazes, you could solve the problem like this:

In [0]:
env = Maze()
env.reset()
actions = ['E','E','S','E','S','S']  # these actions navigate the maze
print(env)

for action in actions:               # loop over actions
    env.step(action)                 # take a step...
    print(env)                       # ...print the maze
    
print('I found the exit!')           # of course you did / you know about mazes

# Is being a maze expert really necessary?

You did good work solving the maze, but at what cost? Your solution won't get you through another equally simple maze, much less solve an entirely different problem. You did what you were asked -- you solved a specific problem -- but that approach loses **generality**. It won't work beyond this one maze.

### It's no longer a maze. It's just a problem.

From here on, let's turn the problem inside-out: forget everything you know about mazes. Instead, ask yourself: how few things could I know and still solve the maze, if I am willing to learn by trial and error? And since that approach avoids **specific** knowledge, does it open the possibility of a **general** solution, that could solve all sorts of problems?

Starting over: you need to solve a problem. You don't know the nature of the problem. What are the fewest things you could know, to attempt a solution, no matter how error-prone or inefficient?


### In general: what class of problems are you attempting to solve?

If you had to list characteristics for a general class of problems like games or a maze, what would you include?

How about:

1. the problem always exists in one (and only one) **state**
2. a chosen **action** causes the problem to transit from one to another state
3. the **current** state of the problem does not depend on any **prior** state
4. the problem signals if you succeed or fail (using a numeric **reward** or **penalty**)
5. the problem signals if the attempt is over (by raising a boolean **done** flag)
6. you are allowed to start over, repeatedly, because: **mistakes**

In general, how many problems could be described that way? Almost every game you've ever played? Landing on the moon? Finding your way through a maze?

### Approach the maze as a general problem

Try the maze again, but this time, forget it's a maze. Don't use the print function. Instead, listen to the feedback provided by the maze environment:


In [0]:
'''
machine learning environments provide feedback, including an
initial state and the dimensions of the state and action spaces.
'''

env = Maze()
obs = env.reset()  # the environment returns an observation of the initial state
print(obs)

OK, the **state** appears to be two-dimensional. What will the environment tell you about the dimensions of the **state space**, which is a description of the dimensions of all possible states?

In [0]:
'''
A 'space' is some collection of discrete or continous possibilities... an egg
carton can be described as a 6x2 space, with an overall dimesion of 12. A chess
board is 8x8, but any of the spaces could contain any piece, so the actual state
space is much larger than 8x8=64. In fact, it is fantastically gigantic.
'''

# what is the state space for the problem?
print(env.state_space())

Looks like the state space is 4x4. That means: no matter what happens, you will be in a bounded space, from (0,0) to (3,3).

What about the available **actions**? How many possible actions are there?

In [0]:
'''
Action spaces can be tricky. For example, in tic-tac-toe, there are 9 actions,
representing an attempt to claim any of the 9 spaces of the board. Here's the
tricky part: there are always 9 actions, regardless of the state of the game.
Putting an 'X' on top of and 'O' is an action -- it's also a bad idea -- but
you would learn that from experience, not from a diminished state space.
'''

# what is the action space for the problem?
print(env.action_space())

Hmmm. The four available actions have cute nicknames: N,S,E,W. The nicknames aren't necessary -- in fact, those will become a nuisance -- but for now, they will help you remember which action is which.

The environment also provides a simpler representation of available actions:

In [0]:
# what is the numeric representation of the action space?
print(env.action_space_n())

That makes sense. There are four actions: 0,1,2,3.

# Take a step, with feedback

Try taking a step or two, while paying attention to the feedback provided by the function step(action). Like many reinforcement learning environments, the maze returns three values when you take a step:
1. an observation of the updated state
2. a numeric reward (or, if negative, a numeric penalty)
3. a 'done' flag (True means 'game over')

Try capturing those return values, and examining each individually:


In [0]:
env = Maze()

# resetting the environment returns the initial state
obs = env.reset()
print('my initial state is', obs)

# taking a step returns an new observation, a reward, and a done flag
print('\nI am about to take an action called E... \n...here goes nothing...\n')
obs, reward, done = env.step('E')
print('my new state is:      ', obs)
print('my reward/penalty is: ', reward)
print('am I done?            ', done)

What exactly do those values represent? Go ahead and print the maze, then take a look.

In [0]:
print(env)

See what happened? Your new state (1,0) represents the position to the east of the entrance. You did not receive a reward -- that's only available at the exit -- nor did you receive a penalty, because nothing went wrong. And is the game over? Not yet.

Trying making a mistake. From your new position, go south, and smack into the blocked square:

In [0]:
print('\nI am about to take an action called S... \n...here goes nothing...\n')
obs, reward, done = env.step('S')
print('my new state is:      ', obs)
print('my reward/penalty is: ', reward)
print('am I done?            ', done)

Get it? Your new state (1,1) is not a good place to be. You received a penalty of -1, and the game ended. That's not all bad. By making a mistake, you found out what not to do. You can always start again.

# Solve the maze with a random walk

What is the minimal algorithm that might solve the maze, now that you recognize the feedback available from step(action)? How about making random moves, until you either succeed, or fail? That's not really machine learning, because you are not taking advantage of experience in each successive attempt, but at least it's a solution.

And: a random walk will show you how to **explore** an environment, without knowing anything specific.

To make things easier, machine learning environments provide a sample() function, which returns a move drawn from a random distribution:


In [0]:
# the environment will sample the action space for you
env = Maze()
print('10 sample actions using nicknames:',[env.sample() for x in range(10)])
print('10 sample actions using numbers:  ',[env.sample_n() for x in range(10)])

Once you have random actions, it's easy enough to take a random walk:

In [0]:
env = Maze()
env.reset()
total_steps = 0
while True:
    # take a random step
    obs,reward,done = env.step(env.sample())
    total_steps += 1
    if done:             # is the game over?
        if reward>0:     #    did you earn a positive reward?
            break        #       sucess! stop walking around
        else:            #    else
            env.reset()  #       failure... try again
            
print('I found my way out, and it only took', total_steps, 'steps...')
print('...I ignored the observations. Who needs those, anyway?')

Well, you found the exit. That's an important result, because it demonstrates that the problem has a solution.

If you can find the exit at random, could you *learn* to find the exit, from experience?

# Machine Learning: the art of learning from mistakes

If you took a first step in the maze, and it went badly, would you take the same step, again?

Probably not (*probably in the mathematical sense... lots of environments change at random; the maze happens to be static*).

Rather than repeating an mistake, you would probably learn from experience.

Here's an example of things going wrong, on the very first step:

In [0]:
# make a maze, then step right out of bounds.
env = Maze()
env.reset()
print('\nI am about to take an action called N... \n...here goes nothing...\n')
obs, reward, done = env.step('N')
print('my new state is:      ', obs)
print('my reward/penalty is: ', reward)
print('am I done?            ', done)

That feedback says: *you are out of bounds, you received a penalty of -1, and the game is over.*

Seems like something worth remembering. How would you keep track of that result, in plain english?

You might say: *hey, on the first step, don't choose N.*

That might help, but it's not quite as general as it could be. Does it really matter if you are on the first step? What happens if you make the same mistake, on the third step?

In [0]:
# try going S,N,N
env = Maze()
env.reset()
print('Here I am at the starting point')
print(env)
print('Am taking the action called S')
env.step('S')
print(env)
print('Am taking the action called N')
env.step('N')
print(env)
print('And why not? Trying N, again')
print('...oops...',env.step('N'))

Now you can see a more general way to describe your experience: **if you are in state(0,0), don't choose action N.** It does not matter how you arrived at (0,0). All that matters is: when your state is (0,0), avoid action N: <code>(0,0)+N->bad</code>.

In more formal terms: a **state-action transition** is the result of taking an action from a given state, causing you to transit to a new state. A notation for an initial move to the south might be:

<code>(0,0)+S->(0,1)</code>

In other words: if you are in state (0,0) and take action S, you will transit to state (0,1):

In [0]:
'''
example of a state-action transition.
from initial state, take action S:
'''
env = Maze()
state = env.reset()
action = 'S'
new_state, _, __ = env.step(action)
print('state-action transition is: ', state, '+', action, '->', new_state)

# Remembering rewards or penalties using a Q-table

In order to learn from experience, you need to accumulate the results of prior state-action transitions. In this case, you will use a 'q-table'. A q-table stores the relative quality of each state-action transition. The idea is simple: for a given state, the quality of each available action is represented by a numeric value. Higher values represent more favorable outcomes; lower values represent (relative) mistakes.

For starters, to represent each space in the maze, you would need a data structure that is 4x4:

In [0]:
import numpy

'''
create an array with one entry for
each square on the maze.
'''
q = numpy.zeros((4,4))
print(q)

That array looks like the maze, but if you wanted to remember <code>(0,0)+N->bad</code>, where would you make that entry? The array includes an element (0,0), but that can only contain one value. If the maze is 4x4, then a 4x4 array could only store the value of a single action, for each state.

What data structure would you use to store one value for each possible **state-action transition**?

If a 4x4 array has one element per state, and each state has 4 available actions (N,S,E,W), how about trying 4x4x4? If you assign some numeric values to the actions (N=0, S=1, E=2, W=3), you could make entries to represent transitions like <code>(0,0)+N</code> (substituting 0 for N would be located at (0,0)+0, or: <code>[0][0][0]</code>): 


In [0]:
import numpy as np
'''
a q-table that represents the quality of state-action transitions
for a problem with 4x4 states and 4 actions
'''

env = Maze()
state = env.reset()
print('initial state', state)
action = 0
print('initial action', action)
obs, reward, done = env.step(action)
print('reward',reward)

q = numpy.zeros((4,4,4))
q[state][action] = reward    # remember the reward/penalty for (0,0)+N
print('\nHere is a q-table with one entry\n')
print(q)

# Visualizing a Q-table
That result may be correct, but it's hard to understand. Try using the maze environment to print the q-table in a more legible form. This format lists the states on the left-hand side, then lays out the actions across the columns. The idea is to clearly show that your one penalty is associated with state (0,0) and action=N:

In [0]:
Maze.print_q(q)

# Store the rewards / penalties from the random walks in the Q-table

Now that you have a place to store rewards and penalties, take random walks through the maze until you find the exit, recording cumulative results in a Q-table:

In [0]:
import numpy as np
'''
take random walks across the maze until you find the exit;
accumulate all rewards / penalties in a q-table.
'''
env = Maze()
state = env.reset()                      # start with the initial state
q = np.zeros((4,4,4))
total_steps = 0

while True:
    action = env.sample_n()              # a numeric action can serve as in index in the q-table
    obs,reward,done = env.step(action)   # take a random step
    q[state][action] += reward           # accumulate the reward or penalty
    state = obs                          # next time, use the new state...
    total_steps += 1                     # keep track of total number of steps
    if done:
        if reward>0:
            break                        # success! you can stop now
        else:
            state = env.reset()          # failure! back to the initial state...

Maze.print_q(q)
print('...it only took', total_steps,'steps!')

Take a look at the Q-table, and try to answer these questions:


*   Why are the largest values negative, and on the first line?
*   Can you find a positive value? From what state was it acheived?
*   Why is there only one positive value?
*   Why does the total number of steps exceed the sum of the values in the table?




# Use the Q-table to avoid past mistakes

What if you tried using the q-table to avoid repeating past mistakes? That algorithm is simple---if an action has resulted in a penalty on a prior attempt, avoid that action---but would that **always** work?

Start by giving it a try:

In [0]:
import numpy as np

env = Maze()
state = env.reset()                      
q = np.zeros((4,4,4))    
total_steps = 0

while True:
    
    action = env.sample_n()       # pick a random action 
    while q[state][action]<0:     # if the action has resulted in a penalty...
        action = env.sample_n()   # ...try something else.
    
    obs,reward,done = env.step(action)   
    q[state][action] += reward           
    total_steps += 1
    state = obs                          
    if done:
        if reward>0:
            break                        
        else:
            state = env.reset()          

Maze.print_q(q)
print('...it only took', total_steps,'steps!')

Hey! That's going in the right direction. However: your approach might fail in an environment where all actions lead to negative penalties, and your best path is to pick the **least bad action**. Also, you might get yourself caught in an infinite loop, if the maze has paths that lead back upon themselves. At this point, you have solve half the problem. You can avoid past penalties. Now you need to find your way *toward* positive rewards.



# numpy to the rescue

Whenever you bump into a slightly thorny issue, chances are that **numpy** has a function to help you out. In this case, you need to find **least bad action**. That will eliminate the possibility that your algorithm fails if all q-values are negative (but some less negative than others). Enter **numpy.argmax**, which returns the *position* of the greatest value in an array, take a look:

In [0]:
import numpy as np

def least_bad(v):
    print('the position of the maximum value is:', np.argmax(v))

least_bad([0,1,2,3,4,5])
least_bad([-5,-4,-3,-2,-1])

r = np.random.random((18))-.5
print('\n',r,'\n')
least_bad(r)

# Learning on the fly: explore v. exploit

How about an approach that combines the random walk with avoiding past mistakes, so you can learn as you go? What if you use random moves to learn by *exploring* the maze, and then use **argmax** to *exploit* what you have learned? Here's a script that flips between exploration and exploitation, use a probability called **explore_rate**:

In [0]:
import numpy as np

env = Maze()
state = env.reset()                      
q = np.zeros((4,4,4))    
total_steps = 0

# experiment with different rates, in interval (0,1)
explore_rate = 0.5

while True:
    
    if np.random.random()<explore_rate:  # roll the dice...
        action = env.sample_n()          # ...explore at random
    else:                                # else...
        action = np.argmax(q[state])     # ...pick best action, from experience
        
    obs,reward,done = env.step(action)   
    q[state][action] += reward           
    total_steps += 1
    state = obs                          
    if done:
        if reward>0:
            break                        
        else:
            state = env.reset()          

Maze.print_q(q)
print('...it only took', total_steps,'steps!')

# Optional: beware the exploding Q-table

Your approach works, but some problems will require that you run your algorithm millions of times, or more. If you update your q-values by **accumulating** numbers, you may find that the values eventually *explode*, meaning: become too large to manage.

To solve the maze, you will make a simplifying assumption in the next section. The remainder of this section describes a more complete, general approach to avoiding large q-values. **This section is optional, you won't need it to get to a complete understanding of Q-learning.**

### OPTIONAL MATH STUFF: the learning rate $(\alpha)$
Many environments have complex reward structures, where the incidence and magnitude of rewards or penalities might be unknown, or might vary. In those cases, you would probably develop q-values by **combining** existing and new rewards using a constant $\alpha$, something like: $q[s,a]=\alpha \times q[s,a]+(1-\alpha) \times reward$ where $0<\alpha<1$. In that general case, $\alpha$ is called the **learning rate**, because $\alpha$ defines the proportion of old versus new information employed to update each value.

For example: using the current algorithm, if you initially take action N a million times, your q-value for <code>(0,0)+N</code> would be -1,000,000. Using a learning rate $\alpha=0.5$, your q-value for <code>(0,0)+N</code> would instead approach, but never equal, -1.0.

### OPTIONAL COMPUTER SCIENCE STUFF: those math people are mixed up
Did you read that **optional math stuff** paragraph? Somebody thinks that a computer can add numbers together forever and approach a limit. Hey! Somebody! Did no one ever mention that computers *don't really store continous floating-point numbers*? Was the whole ones-and-zero's thing lost on you? If you try your example, in Python, the sum will collapse to -1 because of the fundamental limitations of representing floating-point numbers:



In [0]:
'''
this is a side-trip showing how math and CS don't always agree...
it's OK to skip this example.
'''
alpha = 0.5
sum = 0
for n in range(1000000):
    sum = alpha*sum + (1-alpha)*(-1)
    if n%10==0:
        print(n,sum)
    if sum==-1.0:
        print('see? told you so, it only took',n,'loops to equal',sum)
        break

# Simplifying the Q-table

The maze is a **deterministic** problem---the sets of states, actions, state-action transitions, and outcomes never changes. The one-and-only good move is to locate the exit, and all bad moves are equally bad (the game ends). Let's take advantage of that and simplify our q-values, to make the rest of our journey a little easier to understand. You learned a lot by accumulating q-values, because you could observe the **frequency** of each state-action pair. Going forward, you can put those values aside, and simply remember the most recent reward or penalty (you can get away with that because the rewards and penalties are equal and unchanging).

You aren't dong that to throw away information --- your goal is to find a path toward a **future reward**, and this simplifying assumption will make that path easier to see.

In [0]:
import numpy as np

env = Maze()
state = env.reset()                      
q = np.zeros((4,4,4))    
total_steps = 0

explore_rate = 0.5

while True:
    
    if np.random.random()<explore_rate:
        action = env.sample_n()       
    else:                               
        action = np.argmax(q[state])
        
    obs,reward,done = env.step(action)
    q[state][action] = reward  # don't accumulate rewards / just store current value
    total_steps += 1
    state = obs
    if done:
        if reward>0:
            break
        else:
            state = env.reset()

Maze.print_q(q)
print('...it only took', total_steps,'steps!')

The simplified table is now a map of **good, bad, or indifferent** q-values. The remaining problem is: how could you discover the direction *toward* the one-and-only **good** value, starting at state <code>(0,0)</code>?

# What's special about state (2,3)?

If you look back over the q-table values, the only positive reward is recorded from state (2,3). What's so special about state (2,3)?

Take a closer look at state (2,3):

In [0]:
env = Maze()
env.mark = (2,3)
print(env)

To reach the exit, you have to pass through state <code>(2,3)</code>, which is the same as *row 2, column 3*. From that position, moving South solves the maze. Go back to the q-table, and make note: the +1 reward is located at state <code>(2,3)+S</code>.

When you see <code>(2,3)+S-> +1</code>, that implies the following: from state <code>(2,3)</code>, a positive reward is available... just choose action S. Or, let **argmax** choose for you, since +1 is greater than zero.

# What's special about state <code>(1,3)</code>?

Now you are getting someplace. If state <code>(2,3)</code> offers a positive reward, and state <code>(2,3)</code> is available from state <code>(1,3)</code>, what does that imply about state <code>(1,3)</code>?

Take a closer look at state <code>(1,3)</code>:

In [0]:
env = Maze()
env.mark = (1,3)
print(env)

### <code>(1,3)</code> on the first pass, prior to locating exit

If you consider an action from state (1,3) on your first pass through the maze, here are your choices:
* N = blocked square, penalty, game over!
* S = open square, no score, play on
* E = out of bounds, penalty, game over!
* W = open square, no score, play on

### <code>(1,3)</code> on the nth pass, after locating exit at least once
If you have already encountered the exit at least once, and have therefore recorded a positive reward <code>(2,3)+S=+1</code>, then (1,3) looks like this:
* N = blocked square, penalty, game over!
* S = hey... that gets me to a positive reward @ (2,3)
* E = out of bounds, penalty, game over!
* W = open square, no score, play on

Once you have found the exit, <code>(1,3)+S</code> offers a reward on the next step. If you can take an action that leads **toward** a reward, that's better than just wandering around (and way better than smacking into a block or stepping out of bounds).

How could you adjust your the values in your q-table, to recognize that **future** reward?

How about defining the q-value as the sum of (a+b), where:
* a = whatever reward or penalty you received by taking an action
* b = the maximum value available in your new state

You could also write that as:
* a = current reward, based on current experience
* b = future reward, based on past experience

In plain language: *heading toward a good thing is a good thing.*



# Propagating rewards backwards

Try altering your code to peek ahead one step. If there is a future reward available one step forward, add that future reward to the q-value. What happens if you run that until your q-table contains two positive rewards?

In [0]:
import numpy as np

env = Maze()
state = env.reset()                      
q = np.zeros((4,4,4))    
total_steps = 0

explore_rate = 0.5

while True:
    if np.random.random()<explore_rate: 
        action = env.sample_n()         
    else:                               
        action = np.argmax(q[state])
        
    obs,reward,done = env.step(action)

    # when evaluating state+action, consider the max value of future state...
    # ....so q value is (a+b), where a=current reward and b=future reward
    q[state][action] = reward+np.max(q[obs])  # (a+b)
    total_steps += 1
    state = obs                          
    if done:
        if np.sum(q>0)==2: # how does this line manage to count the rewards?
            break                        
        else:
            state = env.reset()          

Maze.print_q(q)
print('...it only took', total_steps,'steps!')

Hey look! Now <code>(1,3)+S=+1</code>.

You are onto something.



# Propogate rewards back to the start

You should probably repeat that process until the positive values work their way back to the start:

In [0]:
import numpy as np

env = Maze()
state = env.reset()                      
q = np.zeros((4,4,4))    
total_steps = 0

explore_rate = 0.5

while True:
    if np.random.random()<explore_rate: 
        action = env.sample_n()         
    else:                               
        action = np.argmax(q[state])
        
    obs,reward,done = env.step(action)
    q[state][action] = reward+np.max(q[obs])
    total_steps += 1
    state = obs                          
    if done:
        if max(q[(0,0)])>0: # the reward has propagated backward
            break                        
        else:
            state = env.reset()          

Maze.print_q(q)
print('...it only took', total_steps,'steps!')

# Plot the positive Q-values

Notice anything familiar about the shape?

It's the path through the maze.

In [0]:
Maze.plot(q)

# Can you choose between two equal rewards?

How would you use the q-table to find the shortest path to the exit of the maze?

Once the table is populated, would that just be a matter of relying on <code>argmax</code> to choose the highest-value action, for any given state?

Before you answer: imagine that you were transported into the **center** of a maze. There is a green line drawn on the floor, that connects the entrance to the exit. You can't see the whole line because it turns a corner in either direction. Would you know which way to go?

Try using the current q-table (warning: this may or may not work, details to follow)



In [0]:
'''
use the existing q-table to walk through the maze,
by always selecting the highest-value action...

NOTE: this code won't run unless the prior code that
      populates the q-table has run during the current 
      session.
'''

env = Maze()
state = env.reset()
done = False
circuit_breaker = 0   # don't run forever!
while not done:
    # use the q-table to select the highest value action for each state
    obs,reward,done = env.step(np.argmax(q[state])) 
    state = obs
    print(env)
    circuit_breaker += 1
    if circuit_breaker > 999:
        break

In [0]:
q[1][2]

If that worked, it was luck; there's a small chance of (complete) failure. If it failed, it probably ran for a very long time. Can you explain why?

Hint: what would happen if you arbitrarily arranged the actions N,S,E,W in a different order, or if the maze had a loop in it? Is there a potential loop in the maze? If you dart back and forth between two spaces... is that a loop?

Run this cell to see the shape of a failed q-table, and ask yourself: if two rewards are avaialable from <code>(0,0)</code>, which action does your algorithm choose? -- and then, which action?

In [0]:
#@title An example of an (unlikely) Q-table failure
import numpy as np

env = Maze()
state = env.reset()                      
q = np.zeros((4,4,4))    
total_steps = 0

explore_rate = 0.5

while True:
    if np.random.random()<explore_rate: 
        action = env.sample_n()         
    else:                               
        action = np.argmax(q[state])
        
    obs,reward,done = env.step(action)
    q[state][action] = reward+np.max(q[obs])
    total_steps += 1
    state = obs                          
    if done:
        if max(q[(1,0)])>0: # the unlikely but possible reward
            break                        
        else:
            state = env.reset()          

Maze.plot(q)

In the example, above, an infinite loop can occur if the random search happens to record a reward at <code>(1,0)</code>. In that case, <code>argmax</code> select the **first** action with a reward of 1, which is **S**, then the **only** action with a reward of 1, which is **N**, then... **S,N,S,N...** forever.

# Degenerate case: a maze with no obstacles

To get a better look at the problem, let's use a maze with no obstacles. That's a *degenerate case* of a maze. Removing the obstacles takes away the distinction of being a maze. Now your environment is indistinguishable from the next broader set of possibilities... it's just an empty room:

In [0]:
'''
when is a maze not a maze? when it's just a room.
'''
env = Maze()
env.remove_blocks()
env.reset()
print(env)

Go ahead and use your algorithm to traverse the empty room. Run this several times, to see the different shapes of the resulting solutions:

In [0]:
import numpy as np

'''
traverse an empty room by removing the blocks from the maze
'''

env = Maze()
env.remove_blocks()    # look ma, no blocks!

q = np.zeros((4,4,4))
total_steps = 0
explore_rate = 0.5

state = env.reset() 
while True:
    if np.random.random()<explore_rate: 
        action = env.sample_n()         
    else:                               
        action = np.argmax(q[state])
 
    obs,reward,done = env.step(action)
    q[state][action] = reward+np.max(q[obs])
    total_steps += 1
    state = obs                          
    if done:
        if max(q[(0,0)])>0:
            break                        
        else:
            state = env.reset()          

Maze.plot(q)
print('...it only took', total_steps,'steps!')

Now try to use your q-table to walk across an empty room. You are very likely to fail. If you happen to succeed... rerun the q-table and try it again.

In [0]:
'''
follow the q-values... very unlikely to succeed!
'''

env = Maze()
env.remove_blocks()
state = env.reset()
done = False
circuit_breaker = 0   # don't run forever!
while not done:
    # use the q-table to select the highest value action for each state
    obs,reward,done = env.step(np.argmax(q[state]))  
    state = obs
    print(env)
    circuit_breaker += 1
    if circuit_breaker > 999:
        break

See? You know there's a reward out there somewhere, but you don't know in which **direction** to go.

# Finding the direction toward a future reward

When you noticed a future reward of +1, you propagated that backward by copying it to each immediately preceding state. By doing that repeatedly, you were able to represent a future reward at every intervening state, including, ultimately, the start. If the ultimate reward for solving a problem is +1, should the first step *toward* that reward also be +1? Or should you use a positive value than can represent both the availability of a future reward, and the direction toward the goal?

Consider these two situations:

* you are on a sidewalk; someone says *there is ice cream at the end of the sidewalk*; which way do you go?

* you are on a staircase; someone says *there is ice cream at the top of the stairs*; which way do you go?

Given those two choices: take the stairs.

### Building a numerical staircase

Take a look at your update to your q-table:

* <code>q[state][action] = reward + np.max(q[obs])</code>

Each q value has two components: <code>reward</code>, which is the immediate result of an action from the **current** state, and <code>max(q[obs])</code>, which is the highest value available from the **next** state. When you notice that the **next** state offers a maximum reward of +1, your code copies the +1 into the current q-value. You are building a sidewalk.

To build a staircase, you need to shrink the value to something less than +1 (but still more than zero). That's like saying *a reward that is one step away is a good thing, but not as good as an immediate reward.*

One simple way to shrink the reward is to multiply by a positive number that is less than 1, for example, $\frac 3 4$:

* <code>q[state][action] = reward + (3/4)*np.max(q[obs])</code>

In that example, your rewards work be: 1.00, 0.75, 0.56, 0.42...

Try it by running this code several times:

In [0]:
import numpy as np

env = Maze()
env.remove_blocks()

q = np.zeros((4,4,4))
total_steps = 0
explore_rate = 0.5

env.reset()
while True:
    if np.random.random()<explore_rate: 
        action = env.sample_n()         
    else:                               
        action = np.argmax(q[state])
        
    obs,reward,done = env.step(action)
    q[state][action] = reward+(3/4)*np.max(q[obs]) # notice the 3/4?
    total_steps += 1
    state = obs                          
    if done:
        if max(q[(0,0)])>0:
            break                        
        else:
            state = env.reset()          
Maze.plot(q)
print('...it only took', total_steps,'steps!')

See all those staircases? Each one leads to the exit.

Now try to use your q-table to select the best action:

In [0]:
'''
follow the q-values... up the stairs, to the exit.
'''

env = Maze()
env.remove_blocks()
state = env.reset()
done = False
while not done:
    # use the q-table to select the highest value action for each state
    obs,reward,done = env.step(np.argmax(q[state]))  
    state = obs
    print(env)

# Putting it all together: up and out of the maze

Now it's easy. You just need to stop removing the blocks from the maze.

Take a look:

In [0]:
import numpy as np

env = Maze()

q = np.zeros((4,4,4))
total_steps = 0
explore_rate = 0.5

env.reset()
while True:
    if np.random.random()<explore_rate: 
        action = env.sample_n()         
    else:                               
        action = np.argmax(q[state])
        
    obs,reward,done = env.step(action)
    q[state][action] = reward+(3/4)*np.max(q[obs])
    total_steps += 1
    state = obs                          
    if done:
        if max(q[(0,0)])>0:
            break                        
        else:
            state = env.reset()          
Maze.plot(q)
print('...it only took', total_steps,'steps!')

In [0]:
'''
follow the q-values... up the stairs, to the exit.
'''

env = Maze()
state = env.reset()
done = False
while not done:
    # use the q-table to select the highest value action for each state
    obs,reward,done = env.step(np.argmax(q[state]))  
    state = obs
    print(env)